In [21]:
import pandas as pd
pd.set_option('display.max_rows', 500)


In [22]:
actions = pd.read_csv("github-action-run.csv.gz")
                          
actions.createdat = pd.to_datetime(actions.createdat * 1000000, utc = True)
actions.updatedat = pd.to_datetime(actions.updatedat * 1000000, utc = True)
actions.firstjobstarted = pd.to_datetime(actions.firstjobstarted * 1000000, utc = True)
actions.lastjobcompleted = pd.to_datetime(actions.lastjobcompleted * 1000000, utc = True)                       

In [23]:
def asf_project(pr):
    name = pr.split("-")[0]
    if name == "incubator":
        return pr.split("-")[1]
    return name

actions["project"] = actions.repo.apply(asf_project)


In [24]:
actions["wall"] = actions.lastjobcompleted - actions.firstjobstarted

In [25]:
actions["wallhours"] = actions.wall.dt.seconds / 60 / 60 

In [26]:
actions["jobhours"] = actions.jobseconds / 60 / 60

In [27]:
actions["month"] = actions.createdat.dt.strftime("%Y-%m")

In [28]:
actions =  actions[actions.month.isin(["2021-03","2021-02"])]
last_month = "2021-03"

## Number of projects using github actions

In [29]:
actions[["id"]].groupby([actions.project,actions.month]).count().reset_index().groupby("month").count()

,project,id
month,,
2021-02,84,84
2021-03,91,91


## Number of jobhours used by month

Maximum available value is 180 * 24 * days == 129600 (30 days) .. 133920 (31 days)

In [30]:
actions[["jobhours"]].groupby([actions.project,actions.month]).count().reset_index().groupby("month").sum()

,jobhours
month,
2021-02,90586
2021-03,102817


## Number of jobs executed by apache projects (last month)

In [31]:
actions[actions.month == last_month][["project","id"]].groupby("project").count().sort_values('id', ascending=False).head(20)

,id
project,
pulsar,22529
arrow,12492
apisix,11196
skywalking,7145
airflow,6659
shardingsphere,5672
camel,5621
nuttx,4108
beam,4042


## User job hours per project (last month)


In [39]:
x = actions[actions.month == last_month][["project","jobhours"]].groupby("project").agg({"jobhours":["sum","mean", "max"]}).sort_values(('jobhours',"sum"), ascending=False)
x["percentage"] = (x["jobhours"]["sum"] / x["jobhours"]["sum"].sum())
x["cumpercentage"] = x.percentage.cumsum()
x.head(20)

jobhours                      percentage cumpercentage
                           sum      mean        max                         
project                                                                     
nuttx             14138.818889  3.441777  17.845833   0.173619      0.173619
pulsar            10785.601944  0.478743   2.011667   0.132443      0.306061
airflow            8305.211111  1.247216  20.768056   0.101984      0.408046
skywalking         6852.736667  0.959095   7.520278   0.084149      0.492194
arrow              6290.633889  0.503573  24.359444   0.077246      0.569440
ozone              5484.444722  4.440846  17.473333   0.067347      0.636787
camel              4241.184722  0.754525  18.681389   0.052080      0.688867
iotdb              4007.576667  1.224061  36.676944   0.049211      0.738078
shardingsphere     2858.329444  0.503937  21.633056   0.035099      0.773177
beam               2782.366111  0.688364  46.451667   0.034166      0.807344
nifi               2380.907500  4.641145  11.313056   0.029237      0.836580
apisix             2342.009722  0.209183  24.139167   0.028759      0.865339
dubbo              1815.491389  1.537249  11.023889   0.022293      0.887632
netbeans           1305.354444  5.159504   6.463611   0.016029      0.903662
pinot              1031.299167  1.507747  16.518889   0.012664      0.916325
mxnet               686.426389  0.921378   2.971111   0.008429      0.924754
iceberg             606.606667  0.629260   6.990278   0.007449      0.932203
maven               527.824167  1.741994  12.020278   0.006481      0.938685
dolphinscheduler    485.171944  0.231475   6.010000   0.005958      0.944643
bookkeeper          431.258056  0.304992   1.787500   0.005296      0.949938

In [40]:
x[x["jobhours"]["sum"] > 1000].to_csv("heavy_users.csv", header=False)